This notebook demonstrates basic usage of firefly_client API with action loistener and controlingn telescope with INDI python client and KStars.

Note that it may be necessary to wait for some cells (like those displaying an image) to complete before executing later cells.

## Setup

Imports for Python 2/3 compatibility

In [68]:
from __future__ import print_function, division, absolute_import

Imports for firefly_client

In [69]:
from firefly_client import FireflyClient

In this example we use the firefly application for the server, use IP host if problems of printing coords

In [70]:
url='http://luke.ipac.caltech.edu:8080/firefly'

Instantiate `FireflyClient`.

In [71]:
fc = FireflyClient(url)

## Download some data

Download a sample image and table.

We use astropy here for convenience, but firefly_client itself does not depend on astropy.

In [72]:
import astropy.utils.data

Download a cutout of a WISE band 1 image.

In [73]:
image_url = ('http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
             '/149/02206a149-w1-int-1b.fits?center=70,20&size=200pix')
filename = astropy.utils.data.download_file(image_url, cache=True, timeout=120)

Download a 2MASS catalog

In [74]:
table_url = ("http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&" +
                                             "QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec)," +
                                             "CIRCLE('J2000',70.0,20.0,0.1))=1")
tablename = astropy.utils.data.download_file(table_url, timeout=120, cache=True)

## Display tables and catalogs

Open a browser to the firefly server in a new tab. The browser open only works when running the notebook locally, otherwise a link is displayed.

In [75]:
localbrowser, browser_url = fc.launch_browser()

Display the web browser link

In [76]:
fc.display_url()

Alternatively, uncomment the lines below to display the browser application in the notebook.

In [77]:
#from IPython.display import IFrame
#print('url: %s' % fc.get_firefly_url())
#IFrame(fc.get_firefly_url(), 1100, 1000)

A local file can be uploaded to the server and then displayed.

In [78]:
imval = fc.upload_file(filename)
status = fc.show_fits(file_on_server=imval, plot_id="wise-cutout", title='WISE Cutout')

`show_fits` can also display an image from a URL

In [79]:
#status = fc.show_fits(file_on_server=None, plot_id="wise-fullimage", 
#                      URL='http://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/6a/02206a' + 
#             '/149/02206a149-w1-int-1b.fits')

A catalog (with coordinates) will overlay the image by default.

In [80]:
file= fc.upload_file(tablename)
status = fc.show_table(file, tbl_id='tablemass', title='My 2MASS Catalog', page_size=50)

An xy plot can be added using the uploaded table data.

Next, add control of the telescope selecting target by clicking (printing coords for now) via an action listener. - to see the widget "print coords" , check "lock-by-click"

In [88]:
import json, subprocess

def pointTelescope(ra,dec):
    # write out json config - to take exposure with INDI client
    configindi = {'star': {'ra':ra, 'dec':dec, 'name':'test'}}
    with open('firefly-wpt-selected.json', 'w') as outfile:  
        json.dump(configindi, outfile)
    subprocess.call(["python3","takeExposure.py","firefly-wpt-selected.json"])
    
def print_coords(event):
    if 'wpt' in event['data']:
       wpt = event['data']['wpt']
       wdata = wpt.split(';')
       ra = float(wdata[0])
       dec = float(wdata[1])
       print('Selected ra={:.6f}, dec={:.6f}'.format(ra,dec))
#       with open('event.json', 'w') as outfile:  
#           json.dump(event['data'], outfile)
       pointTelescope(wdata[0],wdata[1])
    
fc.add_listener(print_coords)

fc.add_extension(ext_type='POINT', plot_id=None, title='Take Exposure!',
                 extension_id='pickit')

{'success': True}

Zoom the full image.

In [ ]:
status = fc.set_zoom('wise-fullimage', 2)

Pan the image to a celestial coordinate.

In [ ]:
status = fc.set_pan('wise-fullimage', 70, 20, coord='J2000')

In [ ]:
fc.display_url()

In [83]:
# Test config json file 
import time
import sys, json
json_text=open('firefly-wpt-selected.json')
config=json.load(json_text)

In [84]:
print(config['star'])
print("RA, DEC of 'star' is ", config['star']['ra'], config['star']['dec'])
print(config['star']['name'])
star = config['star']
print(star['ra'])

{'ra': '69.95358811726766', 'dec': '19.939310133213485', 'name': 'test'}
RA, DEC of 'star' is  69.95358811726766 19.939310133213485
test
69.95358811726766
